In [1]:
%pylab inline
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from sympy import sympify
import sys; sys.path.insert(0, "../"); from utils import *
from models import SympyTorch, PartialDerivativeCalculator, CoeffLearner
from parametric_discovery_pinn import ParametricPINN, BurgerPINN, FinalParametricPINN, RobustPCANN
from pytorch_robust_pca import *
from madgrad import MADGRAD

from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from robust_pde_diff import print_pde, RobustPCA, Robust_LRSTR
from parametric_pde_diff import TrainSGTRidge, create_groups

from scipy.integrate import odeint
from numpy.fft import fft, ifft, fftfreq
from time import time

from pysr import pysr, best

fontsize = 20

Populating the interactive namespace from numpy and matplotlib
Running Python 3.9.6
You can use npar for np.array


In [2]:
data = pickle_load("../parametric_pde_data/parametric_burgers.pkl")

x = data['x']; spatial_dims = x.shape[0]
t = data['t']; time_dims = t.shape[0]

Exact = data['u']
X, T = np.meshgrid(x, t)

# Adding noise
noise_intensity = 0.01
noisy_xt = False

Exact = perturb(Exact, intensity=noise_intensity, noise_type="normal")
print("Perturbed Exact with intensity =", float(noise_intensity))

X_star = np.hstack((to_column_vector(X), to_column_vector(T)))
u_star = to_column_vector(Exact.T)

# Add noise to (x, t) before setting the lb, and ub.
if noisy_xt: 
    print("Noisy (x, t)")
    X_star = perturb(X_star, intensity=noise_intensity, noise_type="normal")
else: print("Clean (x, t)")

# domain bounds
lb = X_star.min(axis=0)
ub = X_star.max(axis=0)

# Sampling training data points
N = 20000
training_idxs = sampling_from_rows(X_star, N, True)
X_train = X_star[training_idxs, :]
u_train = u_star[training_idxs, :]

# Robust PCA
print("Running Robust PCA on (x, t)")
rpca = R_pca_numpy(X_train)
X_train_L, X_train_S = rpca.fit(tol=1e-16, max_iter=20000, iter_print=100, verbose=False)
print('Robust PCA Loss:', mean_squared_error(X_train, X_train_L+X_train_S))
X_train_L = to_tensor(X_train_L, True)
X_train_S = to_tensor(X_train_S, True)

print("Running Robust PCA on u_train")
u_train_L, u_train_S = R_pca_numpy(u_train).fit(tol=1e-16, max_iter=20000, iter_print=100, verbose=False)
u_train_L = to_tensor(u_train_L, False)
u_train_S = to_tensor(u_train_S, False)

del rpca

# to_tensor
X_star = to_tensor(X_star, True)
u_star = to_tensor(u_star, False)
X_train = to_tensor(X_train, True)
u_train = to_tensor(u_train, False)
lb = to_tensor(lb, False)
ub = to_tensor(ub, False)

u_xx_true = 0.1*np.ones(time_dims)
uu_x_true = -1*(1+0.25*np.sin(t))

feature_names = ['u', 'u_x', 'u_xx']

del X_star, u_star, X, T, Exact, data

Loaded from ../parametric_pde_data/parametric_burgers.pkl
Perturbed Exact with intensity = 0.01
Clean (x, t)
Running Robust PCA on (x, t)
Robust PCA Loss: 1.3746377941137367e-32
Running Robust PCA on u_train


In [3]:
pinn = ParametricPINN(scale=False, lb=lb, ub=ub)
print("Loaded the pretrained weights")
pinn.load_state_dict(torch.load("./saved_path_inverse_parametric_burgers/parametric_pinn.pth"))
model = nn.Sequential(pinn.preprocessor_net, pinn.pde_solver_net)

pde_terms = ["u*u_x", "u_xx"]
func_terms = ["-0.1872898*sin(t)-1.0238724", "0.09875935"]
final_burger_pinn = FinalParametricPINN(model=model, pde_terms=pde_terms, func_terms=func_terms, uncert=True, scale=pinn.scale, lb=pinn.lb, ub=pinn.ub)
del pinn

Loaded the pretrained weights


In [4]:
### New implementation ###
class RobustFinalParametricPINN(nn.Module):
    def __init__(self, pinn):
        super(RobustFinalParametricPINN, self).__init__()
        self.pinn = pinn
        self.inp_rpcann = RobustPCANN(beta=0.0, is_beta_trainable=True, inp_dims=2, hidden_dims=50)
        self.out_rpcann = RobustPCANN(beta=0.0, is_beta_trainable=True, inp_dims=1, hidden_dims=50)

    def forward(self, x, t):
        return self.pinn(x, t)

    def loss(self, L, S, y_train_L, y_train_S):
        recov = self.inp_rpcann(L, S, normalize=True)
        return self.pinn.loss(recov[:, 0:1], recov[:, 1:2], self.out_rpcann(y_train_L, y_train_S, normalize=True))
    
### Old implmentation ###
### Old weights might need this class for loading ###
# class RobustFinalParametricPINN(nn.Module):
#     def __init__(self, pinn, beta1=0.0, beta2=0.0, beta3=0.0, is_beta1_trainable=True, is_beta2_trainable=True, is_beta3_trainable=False, hidden_nodes=50):
#         super(RobustFinalParametricPINN, self).__init__()
#         self.pinn = pinn
        
#         self.beta1 = beta1
#         self.beta2 = beta2
#         self.beta3 = beta3
        
#         if is_beta1_trainable: self.beta1 = nn.Parameter(data=torch.tensor([self.beta1]), requires_grad=True)
#         if is_beta2_trainable: self.beta2 = nn.Parameter(data=torch.tensor([self.beta2]), requires_grad=True)
#         if is_beta3_trainable: self.beta3 = nn.Parameter(data=torch.tensor([self.beta3]), requires_grad=True)
        
#         self.proj = nn.Sequential(nn.Linear(2, hidden_nodes), nn.Tanh(), nn.Linear(hidden_nodes, 2), nn.Tanh())
#         self.labels_proj = nn.Sequential(nn.Linear(1, hidden_nodes), nn.Tanh(), nn.Linear(hidden_nodes, 1), nn.Tanh())
        
#     def forward(self, x, t):
#         return self.pinn(x, t)
                
#     def loss(self, L, S, y_train_L, y_train_S):
#         recov = L-self.beta2*self.proj(S)
#         corr = self.labels_proj(y_train_S)
#         return self.pinn.loss(recov[:, 0:1], recov[:, 1:2], y_train_L-self.beta3*corr/torch.norm(corr, p=2))
    
# #     def loss(self, L, S, y_train):
# #         recov = L + S - self.beta2*self.proj(S)
# #         return self.pinn.loss(recov[:, 0:1], recov[:, 1:2], y_train)

In [5]:
robust_buger_pinn = RobustFinalParametricPINN(pinn=final_burger_pinn)

In [6]:
def pcgrad_closure(return_list=False):
    global N, X_train, X_train_L, X_train_S, u_train
    losses = robust_buger_pinn.loss(X_train, X_train_S, u_train, u_train_S)
    updated_grads = []
    
    for i in range(2):
        optimizer.zero_grad()
        losses[i].backward(retain_graph=True)

        g_task = []
        for param in robust_buger_pinn.parameters():
            if param.grad is not None:
                g_task.append(Variable(param.grad.clone(), requires_grad=False))
            else:
                g_task.append(Variable(torch.zeros(param.shape), requires_grad=False))
        # appending the gradients from each task
        updated_grads.append(g_task)

    updated_grads = list(pcgrad.pc_grad_update(updated_grads))[0]
    for idx, param in enumerate(robust_buger_pinn.parameters()):
        param.grad = (updated_grads[0][idx]+updated_grads[1][idx])
        
    if not return_list: return losses[0]+losses[1]
    else: return losses

In [7]:
def finetuning_closure():
    global N, X_train, X_train_L, X_train_S, u_train
    if torch.is_grad_enabled(): f_opt.zero_grad()
    # the solver network only consider the first N samples.
    mse_loss, pde_loss = robust_buger_pinn.loss(X_train, X_train_S, u_train, u_train_S)
    loss = mse_loss + pde_loss
    if loss.requires_grad: loss.backward(retain_graph=True)
    return loss

In [8]:
# optimizer = MADGRAD(robust_buger_pinn.parameters(), lr=1e-6, momentum=0.9)
# robust_buger_pinn.train()
# for i in range(150):
#     optimizer.step(pcgrad_closure)
#     if i%10==0: print(pcgrad_closure(return_list=True))
        
f_opt = torch.optim.LBFGS(robust_buger_pinn.parameters(), lr=1e-1, max_iter=500, max_eval=int(1.25*500), history_size=300, line_search_fn='strong_wolfe')
final_burger_pinn.is_uncert = False
for i in range(50):
    f_opt.step(finetuning_closure)
    if i%10==0: print(finetuning_closure())

tensor(7.1541e-06, grad_fn=<AddBackward0>)
tensor(4.5585e-06, grad_fn=<AddBackward0>)
tensor(4.5585e-06, grad_fn=<AddBackward0>)
tensor(4.5585e-06, grad_fn=<AddBackward0>)
tensor(4.5585e-06, grad_fn=<AddBackward0>)


In [9]:
ests = npar([x.item() for x in robust_buger_pinn.pinn.pdc.parameters()])
ests

array([-1.00097609, -0.24441387,  0.10007171])

In [10]:
grounds = npar([1, 0.25, -0.1])
errs = 100*(abs(ests+grounds)/abs(grounds))
errs.mean(), errs.std()

(0.8012558023134849, 1.0134786013527548)

In [11]:
robust_buger_pinn.inp_rpcann.beta, robust_buger_pinn.out_rpcann.beta

(Parameter containing:
 tensor([0.0005], requires_grad=True),
 Parameter containing:
 tensor([-0.0059], requires_grad=True))

In [12]:
# save(robust_buger_pinn, "./saved_path_inverse_parametric_burgers/noisy_final_doublerpca_parametric_pinn(new).pth")

In [ ]:
# New results

# Noisy Exact & Noisy (x, t) & X_train = X_train - self.beta2*self.proj(S) | beta1 = 0.0
# an attempt with reasonable results
# params: [-0.9991318583488464, -0.23970049619674683, 0.10075154900550842]
# errs: (1.6527215639750146, 1.7654708038535187)

# Noisy Exact & Noisy (x, t) & rpca on (x, t) and rpca on Exact (double rpca)
# Exp NO. 1
# params: [-1.0021382570266724, -0.24129116535186768, 0.09974975883960724]
# errs (1.3158669074376441, 1.5328440955479825)
# (0.0,
#  Parameter containing:
#  tensor([-0.0181], requires_grad=True),
#  Parameter containing:
#  tensor([0.0015], requires_grad=True))
# Exp NO. 2
# array([-1.00135529, -0.24140678,  0.09927386])
# (1.4329850673675555, 1.437619261928569)
# (Parameter containing:
#  tensor([0.0010], requires_grad=True),
#  Parameter containing:
#  tensor([0.0194], requires_grad=True))

# Noisy Exact & Clean (x, t) & rpca on (x, t) and rpca on Exact (double rpca)
# Exp NO. 1
# array([-1.00097609, -0.24441387,  0.10007171])
# (0.8012558023134849, 1.0134786013527548)
# (Parameter containing:
#  tensor([0.0005], requires_grad=True),
#  Parameter containing:
#  tensor([-0.0059], requires_grad=True))

In [ ]:
# Clean Exact and clean (x, t) | final_parametric_pinn.pth
# params: [-1.0013394355773926, -0.2480146884918213, 0.0997583195567131]
# errs: (0.38991620143254785, 0.28918291006992664)

# Noisy Exact and clean (x, t)
# params: [-1.0029208660125732, -0.24306637048721313, 0.09983817487955093]
# errs: (1.075787842273714, 1.2016070382040953)

# Noisy Exact and Noisy (x, t)
# params: [-1.0028020143508911, -0.24078042805194855, 0.10122136771678925]
# errs: (1.729799310366311, 1.4368618683618857)

# Noisy Exact & Clean (x, t) & X_star = X_star_L+X_star_S
# params: [-1.000922441482544, -0.24657735228538513, 0.09988813102245331]
# errs: (0.5243907372156797, 0.5973244500071551)
# Noisy Exact & Noisy (x, t) & X_star = X_star_L+X_star_S
# params: [-1.0046403408050537, -0.24455536901950836, 0.1004636213183403]
# errs: (1.035169263680774, 0.8079990064924926)

In [ ]:
# save(final_burger_pinn, "./saved_path_inverse_parametric_burgers/noisy_final_parametric_pinn.pth")
# save(final_burger_pinn, "./saved_path_inverse_parametric_burgers/noisy2_final_parametric_pinn.pth")